In [13]:
import sys
sys.path.append('..')

In [14]:
from core.dataset import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns

In [15]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [33]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)

In [54]:
cleaned_data = dataset.fillna(0)

In [55]:
cleaned_data.isna().sum().sum()

0

In [56]:
cleaned_data.shape

(14376, 985)

In [57]:
cleaned_data.iloc[:, 984]

0       0
1       0
2       0
3       0
4       0
       ..
1597    7
1598    7
1599    7
1600    7
1601    7
Name: 0, Length: 14376, dtype: int32

In [58]:
X = cleaned_data.iloc[:, [i for i in range(984)]]
y = cleaned_data.iloc[:, 984]

In [59]:
X.isna().sum().sum()

0

In [60]:
y.isna().sum().sum()

0

In [61]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
model = LogisticRegression()

### for logistic regression:
* need to interpolate/remove NaNs

In [64]:
model.fit(X_train, y_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [70]:
pred = model.predict(X_val)

In [71]:
y_val = np.array(y_val)

In [72]:
count = 0
for y_pred, y_true in zip(pred, y_val):
    if y_pred == y_true:
        count += 1
print(count/len(y_val))

0.5104311543810849


In [82]:
model.coef_.shape

(8, 984)

### 8 classes, different set of coefficients for each one, how to generalize them to one set of feature importance?

In [103]:
coefficients = []
for i in model.coef_:
    cdf = pd.DataFrame(i, X.columns, columns=['Coefficients'])
    coefficients.append(cdf)

In [105]:
coefficients[0]

,Coefficients
time_series_1,0.000426
time_series_2,-0.002988
time_series_3,-0.008587
time_series_4,-0.004772
time_series_5,-0.000211
...,...
fold_change_96,0.000578
fold_change_97,0.000636
fold_change_98,0.000595
max_fold_change,0.000550


In [89]:
sorted_CpG_coefficients = coefficients[0].sort_values(by=['Coefficients'])

In [90]:
sorted_CpG_coefficients

,Coefficients
derivatives_1,-0.048896
ipt_1,-0.015126
derivatives_13,-0.012656
derivatives_14,-0.012223
derivatives_12,-0.010847
...,...
derivatives_7,0.013674
derivatives_8,0.015789
derivatives_5,0.021471
derivatives_4,0.039600


In [97]:
test = sorted_CpG_coefficients.iloc[:, 0]

In [117]:
sorted_CpG_coefficients.index.get_loc('integrals_pos_11')

684

#### log reg model does not entirely agree with xgboost ensemble model, will look into this deeper